In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Code"></form>''')

# Project Description

> **DESCRIPTION:** blah blah blah

> **AUTHOR:** Neal Gilmore  
> **DATE:** 27-08-2020

## STEP 1: Environment Preparation (Top)

### 1.1 Import required libraries (Top)

In [2]:
# System libraries
import os
import sys
import time
import warnings
import random

# Standard libraries
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns

# Turn off Depreciation and Future warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

if not sys.warnoptions:
    warnings.simplefilter('ignore')

#%matplotlib inline

print('>> Required libraries imported')

>> Required libraries imported


### 1.2 Prepare for reproducability (Top)

In [3]:
# Set random seed to maintain reproducability
random_state = np.random.seed(42)

# Set the plotting style
#plt.style.use('seaborn-whitegrid')

#sns.set(style="whitegrid", palette="Blues", color_codes=True, font='sans-serif', font_scale=1.5)

# Force display of all columns in dataframes
pd.options.display.max_columns = None

## STEP 2: Data Preparation (Top)

### 2.1 Import data (Top)

In [12]:
# Import the dataset

# set the data working directory
os.chdir(os.path.join(os.getcwd(), "..", "data", "raw"))

# Set the name of the file
filename = 'loan_data_2007_2014.csv'

# Read the data into memory as dask dataframe for speed
%time df_backup = dd.read_csv(filename)

# Create an initial copy of the data for processing
%time df = df_backup.copy()

# Convert dask dataframe to pandas dataframe
%time df = df.compute()

print('\n>> {} data successfully imported'.format(filename))

# set the model working directory
os.chdir(os.path.join(os.getcwd(), "..", "..", "notebooks"))

Wall time: 39.9 ms
Wall time: 998 µs
Wall time: 5.68 s

>> loan_data_2007_2014.csv data successfully imported


### 2.2 Summary statistics (Top)

In [13]:
# Describe the shape of the data
print("\nNumber of rows in the raw data: {}".format(df.shape[0]))
print("Number of columns in the raw data: {}\n".format(df.shape[1]))


Number of rows in the raw data: 466285
Number of columns in the raw data: 75



In [14]:
# Describe the df; Round values to 2 decimals
print(np.round(df.describe(), 2).T)

                                count         mean          std       min  \
Unnamed: 0                   466285.0    233142.00    134605.03      0.00   
id                           466285.0  13079729.91  10893711.62  54734.00   
member_id                    466285.0  14597660.62  11682367.44  70473.00   
loan_amnt                    466285.0     14317.28      8286.51    500.00   
funded_amnt                  466285.0     14291.80      8274.37    500.00   
funded_amnt_inv              466285.0     14222.33      8297.64      0.00   
int_rate                     466285.0        13.83         4.36      5.42   
installment                  466285.0       432.06       243.49     15.67   
annual_inc                   466281.0     73277.38     54963.57   1896.00   
dti                          466285.0        17.22         7.85      0.00   
delinq_2yrs                  466256.0         0.28         0.80      0.00   
inq_last_6mths               466256.0         0.80         1.09      0.00   

In [15]:
# Identify column datatypes
print(df.dtypes)

list(set(df.dtypes.tolist()))

Unnamed: 0            int64
id                    int64
member_id             int64
loan_amnt             int64
funded_amnt           int64
                     ...   
all_util            float64
total_rev_hi_lim    float64
inq_fi              float64
total_cu_tl         float64
inq_last_12m        float64
Length: 75, dtype: object


[dtype('float64'), dtype('int64'), dtype('O')]

In [16]:
df

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.00,0.00,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.00,0.00,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.00,0.00,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.90,766.90,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100358,466280,8598660,1440975,18400,18400,18400.0,60 months,14.47,432.64,C,C2,Financial Advisor,4 years,MORTGAGE,110000.0,Source Verified,Jan-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,773

## STEP 3: Data Pre-processing (Top)¶

## STEP 4: Data Processing (Top)

## STEP 6: Feature Extraction (Top)¶

## STEP 7: Save Processed Data (Top)¶

### 7.1 Drop unnecessary columns (Top)

### 7.2 Save processed data (Top)